# UNDER CONSTRUCTION

Please note this notebook is being developed for ICRA2022, please see earlier for tutorial notebooks that have been completed.

# Tutorial 5: Less Time Wrangling Data, More Time Understanding/Actioning it

**API Version: JuliaLang NavAbilitySDK.jl**

### Problem Statement

Marine operations in harbor and populated river environments will see increased autonomy with major benefits for safety and efficacy, but the navigational environment poses a number of vital challenges.  In developing and testing proof-of-concept systems, four very specific challenges arose:
- GPS data can occasionaly drop out, specifically when maneuvering under bridges or between tall structures;
- Various perception sensors each perform really well with reasonable availability for seeing obstacles or aiding navigation, but no one sensor alone provides a home run solution;
- Practical development testing an operations produces large amounts of recorded robot data, but the data quickly becomes overwhelming and difficult to action;
- How to share / integrate measurement data and navigation between agents (human or autonomous), as well as with shore side operations;

### The Solution

Using the NavAbility Platform, we developed a case study for marine vehicle navigation which direclty addresses these problems.  The underlying SLAM solver is supported by NavAbility as open core software.  Links to the open source code and deeper discussions are available by following links from [the relevant Blog post](https://www.navability.io/2022/02/04/application-example-marine-vehicle-mapping-systems-for-collision-avoidance-and-planning/).

This tutorial below shows how to interact with similar data quickly, efficiently, and with maximum ease.  This tutorial builds on previous tutorials that show how and why factor graphs are so important, as well other tutorials showing some of the robust (non-Gaussian) solver features that are available when building multi-sensor navigation and mapping systems that must combine challenging, ambiguous, and contradictory measurements and data cues.

### The Value (Broader Application)

- By being able to rapidly develop advanced multisensor perception, localization, and mapping ideas drastically reduces the cost and risk of introducing highly focussed autonomy/safety features into existing operations.
- Access to both raw data and actionable SLAM results provides a major platform for machine learning tasks, such as training predictors that have either geometric of semantic relevance -- e.g. learning vehicle behaviors or better classifiers.
- Access to a temporal/spatial database allows various sessions to be brought together with NavAbility's map merging technology -- this will be the discussed more in a future tutorial.

By providing these features through platform and open community, NavAbility is working to give you access to the similar capabilities that marjor brands like Tesla or Amazon are developing privately.

## A Marine SLAM Solution

The marine case study shows benefits value of persistent simultaneous localization and mapping SLAM solutions.
```
Raw Data --> NavAbility Platform SLAM Solving -----> Action / Visualize Results
          ^                                    |
          |                                    |
        Enhance the  <--------------- Deeper processing of data
        factor graph                    with previous resutls
```

## The Data is Alive and Available

> Sketch how the data exists

- Show that the factor graph exists
- Show the size of the factor graph
- Show which variables and factors are in the factor graph

In the sections hereafter, you will learn more about which data is being stored in each of these graph nodes, how the SLAM solution is processed and why that is crucial for multisensor navigation, and how to immediately interact the data on the platform for youself. 


Load the necessary packages or accesing the data:

In [1]:
using NavAbilitySDK

## Read-only Data, `userId=examples@navability.io`

### Robot:Session => MarineASV_ICRA2022:Tutorial5_1000

Point the SDK context to the right location, but selecting the User:Robot:Session

In [2]:
client = NavAbilityHttpsClient()

# you need a unique userId:robotId, and can keep using that across all tutorials
userId = "examples@navability.io"
robotId = "MarineASV_ICRA2022"
sessionId = "Tutorial5_1000"

context = Client(userId, robotId, sessionId)

Client: User=examples@navability.io, Robot=MarineASV_ICRA2022, Session=Tutorial5_1000

### Variables in Session?

To get a sense of scale of the dataset, we can look at the number of variables under this session:

In [5]:
lbls = listVariables(client, context) |> fetch

58-element Vector{String}:
 "x284"
 "x274"
 "x279"
 "x259"
 "x254"
 "x249"
 "x234"
 "x244"
 "x239"
 "x0"
 ⋮
 "x199"
 "x204"
 "x209"
 "x214"
 "x219"
 "x224"
 "x229"
 "x269"
 "x264"

In [12]:
v284 = getVariable(client, context, "x284") |> fetch

Dict{String, Any} with 9 entries:
  "label"        => "x284"
  "solverData"   => Any[Dict{String, Any}("dimval"=>3, "dontmargin"=>false, "Ba…
  "ppes"         => Any[Dict{String, Any}("max"=>Any[1580.4, 118.766, -0.061349…
  "smallData"    => "{}"
  "variableType" => "RoME.Pose2"
  "tags"         => Any["VARIABLE", "POSE"]
  "timestamp"    => "2022-05-22T05:21:51.440Z"
  "_version"     => "0.18.3"
  "solvable"     => 1

### Are the Variables Initialized?

In [25]:
v284["solverData"][1]["initialized"]

true

### Is the Graph solved?

In [23]:
v284["solverData"][1]["solvedCount"]

0

## Large Data Blobs

The solution supports storing large data blobs.

### List Data Entries

Data blobs can be associated with a variable: 

In [36]:
NVA.listDataEntries(client, "x9")

UndefVarError: UndefVarError: listDataEntries not defined

## Bring your own Visualizer (BYOV)

We expect most applications already have a dedicated visualizer techonology choice, and this is good.  The localization and mapping processing here is focussed on providing robust and distributed **results which can be incorporated into existing visualization frameworks**.

### Basic awareness, App Geometric Visualization

To make this tutorial accessible, we show the Marine Case study data with the NavAbility App, which provides a basic 2D visualizer to get a basic understanding of the data:

In [ ]:
MapVizApp(context)

https://app.navability.io/cloud/map/?userId=examples@navability.io&robotStartsWith=MarineASV_ICRA2022&sessionStartsWith=Tutorial5_1000


[![Navigate to Factor Graph](http://www.navability.io/wp-content/uploads/2022/03/geometric_map.png)](https://app.navability.io/cloud/map/?userId=examples@navability.io&robotStartsWith=MarineASV_ICRA2022&sessionStartsWith=Tutorial5_1000)

## Building the Map with/without GPS

A major emphasis of this tutorial is highlighting the importance of robust navigation from all sensors and data sources, while knowing that each sensor has it's own problems and is unlikely to work perfectly 100% of the time -- e.g. GPS fails under bridges when being jammed, cameras fail when the sun or reflections are in the field of view, and radar cross section detections and shadows can vary significatnly.  By embracing non-Gaussian / multimodal data processing challenges head on, we are able to add significant value to the ease and robustness with which the necessary localization / mapping / tracking solutions that need to be developed.

[![Marine Appl Ex](http://www.navability.io/wp-content/uploads/2022/04/MarineRadarAlignFigure-1024x485-1.png)](https://www.navability.io/2022/02/04/application-example-marine-vehicle-mapping-systems-for-collision-avoidance-and-planning/)

Pull two radar sweeps and show a slice of the correlation maps from ScatterAlignPose2 -- discuss.

### Factors in the Session

In [26]:
listFactors(client, context) |> fetch

58-element Vector{String}:
 "x0x4f_843d"
 "x0f_ca58"
 "x279x284f1"
 "x274x279f1"
 "x269x274f1"
 "x264x269f1"
 "x259x264f1"
 "x254x259f1"
 "x249x254f1"
 "x244x249f1"
 ⋮
 "x44x49f1"
 "x39x44f1"
 "x34x39f1"
 "x29x34f1"
 "x24x29f1"
 "x19x24f1"
 "x14x19f1"
 "x9x14f1"
 "x4x9f1"

We can look a bit closer at one of the radar odometry factors:

In [29]:
f_x4x9 = getFactor(client, context, "x4x9f1") |> fetch

Dict{String, Any} with 8 entries:
  "label"                 => "x4x9f1"
  "_variableOrderSymbols" => Any["x4", "x9"]
  "data"                  => "eyJlbGltaW5hdGVkIjpmYWxzZSwicG90ZW50aWFsdXNlZCI6Z…
  "tags"                  => Any["FACTOR"]
  "timestamp"             => "2022-05-24T05:34:17.016Z"
  "_version"              => "0.18.3"
  "fnctype"               => "ScatterAlignPose2"
  "solvable"              => 1

The results from the line above shows a "data" value which in this particular case is a `base64` encoded string.  That field contains the data for a `Caesar.ScatterAlignPose2` factor -- also shows by:

In [30]:
f_x4x9["fnctype"]

"ScatterAlignPose2"

## Find / Track Obstacles with NavAbilitySDK

### Large Data Blobs




### Find Boats in Camera Data

- Pull image data,
- Process camera image with user side needs,

### Track Boats in the Radar Data

### Combining Camera / Radar info Back into Factor Graph

- Dummy stub for uploading data back (not allowed under guest, please sign up for write access to a session)
- Pull all this data into the factor graph and ask for a new solve

### We Baked one Earlier

- Show a piece of the factor graph where a target object boat is being tracked by both camera and radar factors in the graph,
- Visualize the trajectory of the target boat, which was estimated from our vehicle regardless of whether GPS was working or not!

### Query spatial relationships,

- What is here there, or nearby
- What might see this target


## Next Steps

If you have interest in NavAbility and what our technology can do, please reach out via [Slack](https://join.slack.com/t/caesarjl/shared_invite/zt-ucs06bwg-y2tEbddwX1vR18MASnOLsw) or [info@navability.io](info@navability.io).